In [9]:
import os, sys
sys.path.append(os.getcwd())

import numpy as np
import tensorflow as tf
import scipy.misc
import imageio
from imageio import imsave, imread
#from scipy.misc import imsave, imread

In [12]:
import tflib as lib
import tflib.sampling_loop
import tflib.ops.kl_unit_gaussian
import tflib.ops.kl_gaussian_gaussian
import tflib.ops.conv2d
import tflib.ops.linear
import tflib.ops.batchnorm
import tflib.ops.embedding

import os
import urllib
import gzip
import cPickle as pickle

import time
import functools
import random

DATASET = 'mnist_256' # mnist_256

In [15]:
classindices = [1,1]
hi = np.multiply(0.5, classindices) + np.multiply(0.5, classindices)
imsave('/Users/wildflowerlyi/Desktop/' + 'samples_{}.txt', hi)

ValueError: Image must be 2D (grayscale, RGB, or RGBA).

In [16]:
classindices = np.array([[0,6],[3,8]])
classindices[0,]
classindices.shape[0]
idx = np.zeros(classindices.shape[0])
for classnums in xrange(classindices.shape[0]):
    idx[classnums] = 1

In [17]:
# Test how to index things
y_train = [4, 0, 1, 2, 3, 4]
idx = np.where(np.equal(4,y_train))
print(idx[0])
y_train = np.array(y_train)
print(y_train[idx[0]])

[0 5]
[4 4]


In [18]:
# How data is loaded into PixelVAE
data = urllib.urlretrieve('http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz', 'mnist_test.gz')[0]

with gzip.open(data, 'rb') as f:
    train_data, dev_data, test_data = pickle.load(f)

In [19]:
print(train_data[0][1]) # pixels are in the range 0-1

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [20]:
def mnist_reshape(data):
    
    images, targets = data
    images = images.astype('float32')
    images = images.reshape(-1, 784)
    images2 = (images*(256-1e-8)).astype('int32')
    
    targets = targets.astype('float32')

    return(np.copy(images2), np.copy(targets))

In [21]:
train_data = mnist_reshape(train_data)

In [22]:
index = np.random.randint(0, len(train_data[1]), 2)

In [23]:
print(train_data[0][index[0]]) # pixels are now integers

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  13 219 214  69   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  61 184 204 136  33   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  51 248 254  87   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   3 179 254 232  72   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 132 254 254  43   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  28 157 252 252 164   2   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   7 218 254 254 179   0   0   0   0   0   0   0   

In [24]:
image_samples = train_data[0][index]

In [25]:
label_samples = train_data[1][index]

In [26]:
new_target = (np.mean(label_samples))

In [27]:
# save images as uint8 arrays
image_samples_int8  = image_samples.astype(np.uint8)

In [28]:
samples_new = image_samples, label_samples

In [29]:
sample1 = image_samples[1], label_samples[1]

In [85]:
#print(samples_new)

In [84]:
#print(samples_new[0][0]) # before converting to uint8

In [83]:
#print(image_samples_int8[0]) # after converting to uint8

In [32]:
# Read in png file and target as arrays
testimage = imread('/Users/wildflowerlyi/Desktop/linear_interpolations_[a_mnist/encoded_reconsamples_06_0.png')
print(testimage.shape)
testtarget = np.array([1.5])

OSError: No such file: '/Users/wildflowerlyi/Desktop/linear_interpolations_mnist/encoded_reconsamples_06_0.png'

In [22]:
# Import Keras
import keras
from keras.datasets import mnist, cifar10
import imageio
from tempfile import TemporaryFile

(x_train, y_train), (x_test, y_test) = mnist.load_data()
#(x_traincifar, y_traincifar), (x_testcifar, y_testcifar) = cifar10.load_data()

In [10]:
new_xtrain = x_train.reshape(-1, 1, 28, 28)
new_xtrain.shape
new_ytrain = y_train.reshape(-1, 1)
new_ytrain.shape
new_xtrain.shape
new_xtrain[0,]

array([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
           0

In [23]:
# Test how to index things
idx = np.where(np.equal(4,new_ytrain))
print(idx[0])
new_ytrain = np.array(new_ytrain)
new_xtrain = np.array(new_xtrain)
new_xtrain[idx[0]].shape
x2 = new_xtrain[idx[0]][1].reshape(-1, 1, 28, 28)
x2.shape
new_xtrain.shape

[    2     9    20 ..., 59943 59951 59975]


(60000, 1, 28, 28)

In [26]:
# Save a generated image and its corresponding target as a tuple
outfile = TemporaryFile()
#np.savez(outfile,image = testimage, label = testtarget)
np.save(outfile, image = testimage)
np.save()

NameError: name 'testimage' is not defined

In [56]:
outfile.seek(0)
npzfile = np.load(outfile)

In [57]:
npzfile.files

['image', 'label']

In [95]:
# Reshape the image
# If MNIST
newimage = sample1[0].reshape(-1, 28, 28)
# If CIFAR
#newimage = sample1[0].reshape(-1, 3, 32, 32)

In [93]:
# Add image and target to dataset
x_train_aug = np.append(x_train, newimage2, axis=0)
y_train_aug = np.append(y_train, sample1[1])

In [6]:
np.save('/Users/wildflowerlyi/Desktop/test', x_train_aug)

NameError: name 'x_train_aug' is not defined

In [99]:
x_augmentations_arrays = np.load('/Users/wildflowerlyi/Desktop/test.npy')

In [ ]:
# SAVING AUGMENTATIONS: FINAL
# (0) Initialize image array
# (1) Produce image array
# (2) Add image array to those before
# (3) Save final array at end of for loop

In [ ]:
# Insert this at line 798
x_augmentations_array = []

# Insert this line under the sample generation loop in line 823
print "Appending image array to those previously generated"
    x_augmentations_array = np.append(x_augmentations_array, samples, axis=0)

# Insert this line after all images have been added, after line 834 (make sure not indented)
np.save('x_augmentations_array', x_augmentations_array)

In [ ]:
# READING IN AUGMENTATIONS TO TRAIN NEURAL NETWORKS: FINAL
# (1) Read in x and y augmentations as new_x and new_y
# (2) Reshape new_x as = new_x.reshape(-1, 28, 28) or new_y = new_y.reshape(-1, 3, 32, 32)
#     x_augmented = np.append(x_train, new_x, axis=0)
#     y_augmented = np.append(y_train, new_y, axis=0)

# Add the below text at the beginning of train_cifar10 files in each directory. Make sure the augmented files are 
# in each of the folders resnet_cifar, resnet_wide, resnet_densenet

In [ ]:
# Import Keras
import keras
from keras.datasets import mnist, cifar10
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_traincifar, y_traincifar), (x_testcifar, y_testcifar) = cifar10.load_data()

# Load augmented arrays
new_x = np.load('/Users/wildflowerlyi/Desktop/Github/NN_calibration/scripts/resnet_cifar/x_augmentation_array.npy')
new_y = np.load('/Users/wildflowerlyi/Desktop/Github/NN_calibration/scripts/resnet_cifar/y_augmentation_array.npy')

# Reshape from (H,W,CH) to (CH,H,W)
# new_x = np.transpose(new_x, (2, 1, 0)) # Only for CIFAR

# Add images and targets to original dataset
x_train_aug = np.append(x_train, new_x, axis=0)
y_train_aug = np.append(y_train, new_y, axis=0)